# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
print(data[['call','race']].info())
data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 2 columns):
call    4870 non-null float32
race    4870 non-null object
dtypes: float32(1), object(1)
memory usage: 95.1+ KB
None


,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

### 1) What test is appropriate for this problem? Does CLT apply?
* For this problem it would be best to use a bootstrap hypothesis test on the differences of means between the average number of black and white sounding names that got called back. 
* The CLT definitely does apply as we can see from above that there are 4870 observations, which is a large sample size. Also, we are going to be comparing the averages of many bootstrap samples, which will end up having a normal distribution.

### 2) What are the null and alternate hypotheses?
* **Null Hypothesis**: The average number of callbacks for black sounding names is equal to that of white sounding names.
* **Alternate Hypothesis**: The average number of callbacks depends on whether the name is white or black sounding

### 3) Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [5]:
w = data[data.race=='w']
b = data[data.race=='b']

num_call = data.call.sum()
num_wh = w.call.sum()
num_bl = b.call.sum()
print('# of total sounding callbacks:',num_call)
print('# of white sounding callbacks:',num_wh)
print('# of black sounding callbacks:',num_bl)

# of total sounding callbacks: 392.0
# of white sounding callbacks: 235.0
# of black sounding callbacks: 157.0


In [6]:
total = data.call
white = w.call
black = b.call

#take the mean of all temperatures
call_mean = data.call.mean()
wh_mean = white.mean()
bl_mean = black.mean()

#find the difference of mean betwen the male and female observations
diff_mean = wh_mean - bl_mean
print('Percent of total callbacks:',call_mean)
print('Percent of white callbacks:',wh_mean)
print('Percent of black callbacks:',bl_mean)
print('Difference of means between male and female:',diff_mean)

Percent of total callbacks: 0.08049281686544418
Percent of white callbacks: 0.09650924056768417
Percent of black callbacks: 0.0644763857126236
Difference of means between male and female: 0.03203285485506058


In [7]:
#obtain zscore, margin of error and confidence interval
std_err = ((white.std()**2 / len(w)) + (black.std()**2/len(b)))**.5
z = stats.norm.ppf(.975)
margin_error = z*std_err
conf_int = (diff_mean-margin_error, diff_mean+margin_error)

print('Standard error:',std_err)
print('zscore at 95% confidence',z)
print('Margin of Error for the sample:',margin_error)
print('95% Confidence Interval:',conf_int)


Standard error: 0.007784906919813793
zscore at 95% confidence 1.95996398454
Margin of Error for the sample: 0.0152581371858
95% Confidence Interval: (0.016774717669228896, 0.047290992040892259)


In [8]:
def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1,data2))

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2

def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1,data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1,perm_sample_2)

    return perm_replicates

def diff_of_means(data_1, data_2):
    """Difference in means of two arrays."""

    # The difference of means of data_1, data_2: diff
    diff = np.mean(data_1) - np.mean(data_2)

    return diff

In [9]:
# Draw 10,000 permutation replicates: perm_replicates
perm_replicates = draw_perm_reps(white, black, diff_of_means, size=10000)

# Compute p-value: p
p = np.sum(perm_replicates >= diff_mean) / len(perm_replicates)

# Print the result
print('p-value =', p)

p-value = 0.0


### 4) Write a story describing the statistical significance in the context or the original problem.

The lack of equality in the work place has been an issue in the United States since the workforce even started to become integrated in the 1900's. To see whether or not racial discrimination exists in the hiring process, we analyzed the results of data collected from interviews of randomly selected applicants. The resumes were identical, except for the names being black or white sounding names. The results were pretty apparent. Just by looking at the difference in the average number of callbacks between the 3 races, we see that percentage of white names called back was 3.2% higher than the percentage of black name. Taking our analysis further, I test the null hypothesis that the callback percentages were equal. I drew 10000 permutation replicates of each type of name to see if these results were by chance. The returned p-value ended up being 0. This shows that the difference in the proportion of white names and black names being hired is statistically significant. This can be traced to racial discrimination since everything but the name were the same.


### 5) Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

The analysis doesn't necesarrily mean that race is the *most* important, it just means that it *is*  important. We don't know where the interviewers were located and where they are from. The results might be very different in different cities. More liberal cities will probably exhibit a lot more equality than many southern/conservative cities. This would definitely be my next step, is to identify the cities or states where the discrimination is more apparent by testing specific locations. 